<a href="https://colab.research.google.com/github/aalemi97/AFRetryRequest/blob/master/Visualizers/UnitsTour.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Colab Configuration

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


# Units Tour

In [6]:
import numpy as np
import pandas as pd
import folium
from folium.plugins import HeatMap
from folium.plugins import HeatMapWithTime

class TourVisualizer(object):
  agents = None
  zones = None

  def __init__(self, agentsSummaryPath, zonesInfoPath):
    self.agents = pd.read_pickle(agentsSummaryPath)
    self.zones = pd.read_csv(zonesInfoPath)
    super().__init__()

  def getUnitIDs(self):
    return self.agents["Unit ID"].unique()

  def getDates(self):
    return self.agents["Date"].unique()

  def showHeatMap(self, unitID, date):
    row = self.agents.loc[(self.agents["Unit ID"] == unitID) &
                          (self.agents["Date"] == date)]
    if not row.empty:
      spots = list(map(lambda x: int(x),
                       row["Observed Zones ID"].values[0][1:-1].split(", ")))
      uniqueSpots = set(spots)
      lat = []
      lon = []
      observations = []
      for spot in uniqueSpots:
        observations.append(spots.count(spot))
        point = self.zones.loc[self.zones["Zone Number"] == spot,
                            "Center Point"].values[0]
        point = point.split(" ")
        lon.append(float(point[1][1:]))
        lat.append(float(point[2][:-1]))
      
      df = pd.DataFrame(columns=["Lat", "Lon", "Observations"])
      df["Lat"] = lat
      df["Lon"] = lon
      df["Observations"] = observations
    else:
      return None, None

    hm = folium.Map(location=[51.0447, -114.0719], zoom_start=11)

    HeatMap(df,
            min_opacity=0.4,
            blur = 18).add_to(folium.FeatureGroup(name='Heat Map').add_to(hm))

    folium.LayerControl().add_to(hm)
    return df, hm

  def showTimeHeatMap(self, unitID):
    unit = self.agents.loc[
        (self.agents["Unit ID"] == unitID)].sort_values(by=["Date"])

    data = []

    if not unit.empty:
      dates = unit["Date"].unique().tolist()
      for _, row in unit.iterrows():
        spots = row["Observed Zones ID"]
        uniqueSpots = set(spots)
        temp = []
      
        for spot in uniqueSpots:
          observation = spots.count(spot)
          point = self.zones.loc[self.zones["Zone Number"] == spot,
                              "Center Point"].values[0]
          point = point.split(" ")
          lon = float(point[1][1:])
          lat = float(point[2][:-1])
          temp.append([lat, lon, observation])
        
        data.append(temp)
      
      hmt = folium.Map(location=[51.0447, -114.0719],
                       zoom_start=11,
                       control_scale=True)
      
      HeatMapWithTime(data,
                      index=dates,
                      auto_play=True,
                      use_local_extrema=True).add_to(hmt)

      legend_html = \
      '''
        <div style="position:fixed;
                    bottom: 50px; right: 50px;
                    z-index:1000;
                    padding: 10px;
                    background-color: white;
                    opacity: 0.7;
                    border-radius: 5px;
                    font-size: 12px;">
        <strong>Legend</strong><br>
        <div style="background-color: blue; width: 20px; height: 10px; display: inline-block;"></div>
        Least Observation<br>
        <div style="background-color: yellow; width: 20px; height: 10px; display: inline-block;"></div>
        Medium Observation<br>
        <div style="background-color: red; width: 20px; height: 10px; display: inline-block;"></div>
        High Observation
        </div>

      '''
      hmt.get_root().html.add_child(folium.Element(legend_html))

      return data, hmt
    else:
      return None, None

  def showAllTimeHeatMap(self):

    m = folium.Map(location=[51.0447, -114.0719],
                   zoom_start=11,
                   control_scale=True)

    ids = self.getUnitIDs()
    for unitID in ids:
      unit = self.agents.loc[
          (self.agents["Unit ID"] == unitID)].sort_values(by=["Date"])

      data = []

      if not unit.empty:
        dates = unit["Date"].unique().tolist()
        dates = list(map(lambda x: f"{unitID} {x}", dates))
        for _, row in unit.iterrows():
          spots = row["Observed Zones ID"]
          uniqueSpots = set(spots)
          temp = []
        
          for spot in uniqueSpots:
            observation = spots.count(spot)
            weight = observation/len(spots)
            
            point = self.zones.loc[self.zones["Zone Number"] == spot,
                                "Center Point"].values
            if len(point) >= 1:
              point = point [0]
              point = point.split(" ")
              lon = float(point[1][1:])
              lat = float(point[2][:-1])
              temp.append([lat, lon, weight])
          
          data.append(temp)

        hmt = HeatMapWithTime(data,
                              index=dates,
                              name=f"{unitID}",
                              auto_play=False,
                              use_local_extrema=True,
                              position="bottomleft",
                              overlay=True,
                              control=True,
                              show=False)
        hmt.add_to(m)
    legend_html = \
    '''
      <div style="position:fixed;
                  bottom: 50px; right: 50px;
                  z-index:1000;
                  padding: 10px;
                  background-color: white;
                  opacity: 0.7;
                  border-radius: 5px;
                  font-size: 12px;">
      <strong>Legend</strong><br>
      <div style="background-color: blue; width: 20px; height: 10px; display: inline-block;"></div>
      Least Observation<br>
      <div style="background-color: yellow; width: 20px; height: 10px; display: inline-block;"></div>
      Medium Observation<br>
      <div style="background-color: red; width: 20px; height: 10px; display: inline-block;"></div>
      High Observation
      </div>

    '''
    m.get_root().html.add_child(folium.Element(legend_html))
    folium.LayerControl().add_to(m)
    return m

In [7]:
visualizer = TourVisualizer(
    agentsSummaryPath="/content/drive/MyDrive/UCalgary/Thesis/CPA Data/Visualizations/Units/UnitsDF.pkl",
    zonesInfoPath="/content/drive/MyDrive/UCalgary/Thesis/CPA Data/Zones/FinalZonesDF.csv")

In [8]:
m = visualizer.showAllTimeHeatMap()

In [10]:
m.save("/content/drive/MyDrive/UCalgary/Thesis/CPA Data/Visualizations/Units/AllTimeHeayMap.html")